In [ ]:
# run(`conda create --channel conda-forge --channel bioconda --channel defaults --strict-channel-priority --name blast blast`)
# run(`conda create --channel conda-forge --channel bioconda --channel defaults --strict-channel-priority --name taxonkit taxonkit`)
# run(`conda create --channel conda-forge --channel bioconda --channel defaults --strict-channel-priority --name minimap2 minimap2`)

In [27]:
# don't try and install plotting libraries without this
# can set in ~/.local/share/jupyter/kernels/
@assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
pkgs = [
    "Revise",
    "DataFrames",
    "uCSV",
    "StatsPlots",
    "StatsBase",
    "FreqTables",
    "Conda",
    "ProgressMeter",
    "PrettyTables",
    "Distances",
    "Statistics",
    "Kmers",
    "Colors",
    "FASTX"
]
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
import Mycelia

In [2]:
base_dir = dirname(pwd())
data_dir = joinpath(base_dir, "data")
results_dir = joinpath(data_dir, "results")

# load in metadata
metadata_dir = joinpath(dirname(pwd()), "metadata")

exposome_environmental_data = DataFrames.DataFrame(uCSV.read(
    joinpath(metadata_dir, "metadata_exposome.rds.tsv"),
    delim='\t',
    header=1,
    typedetectrows=300
))

joint_sample_metadata = DataFrames.DataFrame(uCSV.read(
    joinpath(metadata_dir, "exposome/joint_sample_metadata.tsv"),
    delim='\t',
    header=1,
    typedetectrows=300
))

@assert joint_sample_metadata[!, "Library Name"] == joint_sample_metadata[!, "LibraryName"]

joint_metadata = DataFrames.innerjoin(
    joint_sample_metadata,
    exposome_environmental_data,
    on="Library Name" => "samplenames")

sample_directories = joinpath.(data_dir, "SRA", joint_metadata[!, "Run"])

┌ Warning: Large values for `typedetectrows` will reduce performance. Consider manually declaring the types of columns using the `types` argument instead.
└ @ uCSV ~/.julia/packages/uCSV/goLRH/src/read.jl:213
┌ Warning: Large values for `typedetectrows` will reduce performance. Consider manually declaring the types of columns using the `types` argument instead.
└ @ uCSV ~/.julia/packages/uCSV/goLRH/src/read.jl:213


213-element Vector{String}:
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/SRA/SRR6399706"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/SRA/SRR6399707"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/SRA/SRR6399708"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/SRA/SRR6399526"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/SRA/SRR6399525"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/SRA/SRR6399520"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/SRA/SRR6399972"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/SRA/SRR6399522"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/SRA/SRR6399521"
 "/oak/stanford/scg/lab_mps

In [3]:
viral_contigs_by_tool = Dict{String, Dict{String, Set{String}}}()

Dict{String, Dict{String, Set{String}}}()

In [4]:
joint_genomad_results = DataFrames.DataFrame()
# sample_directory = first(sample_directories)
ProgressMeter.@showprogress for sample_directory in sample_directories
    genomad_virus_summary = joinpath(sample_directory, "genomad", "final.contigs.fastg.gfa_summary", "final.contigs.fastg.gfa_virus_summary.tsv")
    genomad_results = DataFrames.DataFrame(uCSV.read(genomad_virus_summary, delim='\t', header=1, typedetectrows=100)...)
    genomad_results[!, "sample_id"] .= basename(sample_directory)
    append!(joint_genomad_results, genomad_results, promote=true)
end
joint_genomad_results[!, "seq_name"] = string.(joint_genomad_results[!, "seq_name"])
joint_genomad_results

Progress: 100%|█████████████████████████████████████████| Time: 0:00:08


Row,seq_name,length,topology,coordinates,n_genes,genetic_code,virus_score,fdr,n_hallmarks,marker_enrichment,taxonomy,sample_id
,String,Int64,String,String,Int64,Int64,Float64,String,Int64,Float64,String,String
1,24575,5527,DTR,NA,8,11,0.982,NA,2,10.3097,Viruses;Monodnaviria;Sangervirae;Phixviricota;Malgrandaviricetes;Petitvirales;Microviridae,SRR6399706
2,29271,2611,No terminal repeats,NA,4,11,0.9786,NA,0,6.471,Viruses;Duplodnaviria;Heunggongvirae;Uroviricota;Caudoviricetes,SRR6399706
3,25956,4092,No terminal repeats,NA,9,11,0.9648,NA,0,5.1548,Viruses;Duplodnaviria;Heunggongvirae;Uroviricota;Caudoviricetes,SRR6399706
4,29340,2590,No terminal repeats,NA,4,11,0.9562,NA,0,6.6376,Viruses;Duplodnaviria;Heunggongvirae;Uroviricota;Caudoviricetes,SRR6399706
5,17257,658,No terminal repeats,NA,2,11,0.9522,NA,2,2.787,Viruses;Duplodnaviria;Heunggongvirae;Uroviricota;Caudoviricetes,SRR6399706
6,28830,3412,No terminal repeats,NA,7,11,0.947,NA,0,2.5057,Viruses;Duplodnaviria;Heunggongvirae;Uroviricota;Caudoviricetes,SRR6399706
7,23428,3277,No terminal repeats,NA,9,11,0.9386,NA,0,7.8593,Viruses;Duplodnaviria;Heunggongvirae;Uroviricota;Caudoviricetes,SRR6399706
8,9279,4961,No terminal repeats,NA,1,11,0.9223,NA,0,1.7183,Viruses;Varidnaviria;Bamfordvirae;Preplasmiviricota;Tectiliviricetes;Autolykiviridae,SRR6399706
9,24694,2652,No terminal repeats,NA,8,11,0.9201,NA,0,3.4366,Viruses;Duplodnaviria;Heunggongvirae;Uroviricota;Caudoviricetes,SRR6399706


In [5]:
viral_contigs_by_tool["genomad"] = Dict{String, Set{String}}()
for gdf in DataFrames.groupby(joint_genomad_results, "sample_id")
    sample_id = gdf[1, "sample_id"]
    viral_contigs_by_tool["genomad"][sample_id] = Set()
    for row in DataFrames.eachrow(gdf)
        push!(viral_contigs_by_tool["genomad"][sample_id], row["seq_name"])
    end
end
viral_contigs_by_tool["genomad"]

Dict{String, Set{String}} with 213 entries:
  "SRR6399620" => Set(["25060", "58575", "21490", "57482", "33110", "43681", "1…
  "SRR6399522" => Set(["35351", "10657", "39489", "35171", "30061", "39970", "2…
  "SRR6399796" => Set(["4659", "15743", "15859", "2360", "15763", "12792", "112…
  "SRR6399686" => Set(["28931", "20278", "38091", "17527", "41020", "45034", "2…
  "SRR6399461" => Set(["6086", "9587", "4230", "20677", "1651", "14090", "3973"…
  "SRR6399715" => Set(["34950", "32420", "26563", "18322", "25183", "33125", "2…
  "SRR6399802" => Set(["34461", "5822", "23752", "28306", "7925", "22826", "302…
  "SRR6399613" => Set(["12960", "9620", "3747", "1848", "8951", "9945", "10214"…
  "SRR6399716" => Set(["9687", "8891", "540", "6327", "15778", "1750", "263", "…
  "SRR6399615" => Set(["3478", "908", "3343", "9449", "6666", "8771", "4800", "…
  "SRR6399464" => Set(["36948", "5790", "34631", "52460", "53243", "22797", "74…
  "SRR6400018" => Set(["4304", "35674", "36966", "17750", "30029"

In [6]:
blast_task = "megablast"

"megablast"

In [7]:
db = "nt_viruses"

"nt_viruses"

In [8]:
# load in metadata
metadata_dir = joinpath(dirname(pwd()), "metadata")

exposome_environmental_data = DataFrames.DataFrame(uCSV.read(
    joinpath(metadata_dir, "metadata_exposome.rds.tsv"),
    delim='\t',
    header=1,
    typedetectrows=300
))

joint_sample_metadata = DataFrames.DataFrame(uCSV.read(
    joinpath(metadata_dir, "exposome/joint_sample_metadata.tsv"),
    delim='\t',
    header=1,
    typedetectrows=300
))

@assert joint_sample_metadata[!, "Library Name"] == joint_sample_metadata[!, "LibraryName"]

joint_metadata = DataFrames.innerjoin(
    joint_sample_metadata,
    exposome_environmental_data,
    on="Library Name" => "samplenames")

run_ids = sort(joint_metadata[!, "Run"])

sample_paths = joinpath.(data_dir, "SRA", run_ids)

┌ Warning: Large values for `typedetectrows` will reduce performance. Consider manually declaring the types of columns using the `types` argument instead.
└ @ uCSV ~/.julia/packages/uCSV/goLRH/src/read.jl:213
┌ Warning: Large values for `typedetectrows` will reduce performance. Consider manually declaring the types of columns using the `types` argument instead.
└ @ uCSV ~/.julia/packages/uCSV/goLRH/src/read.jl:213


213-element Vector{String}:
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/SRA/SRR6399459"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/SRA/SRR6399460"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/SRA/SRR6399461"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/SRA/SRR6399462"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/SRA/SRR6399464"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/SRA/SRR6399485"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/SRA/SRR6399486"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/SRA/SRR6399487"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/SRA/SRR6399488"
 "/oak/stanford/scg/lab_mps

In [9]:
# NCBI host metadata
ncbi_metadata_file = joinpath(dirname(pwd()), "metadata", "NCBI-virus-refseq.transformed.tsv")
ncbi_host_metadata = DataFrames.DataFrame(uCSV.read(ncbi_metadata_file, header=1, delim='\t', encodings=Dict("false" => false, "true" => true)))

# ICTV host metadata
ictv_metadata_file = joinpath(dirname(pwd()), "metadata", "VMR_MSL38_v1 - VMR MSL38 v1.transformed.tsv")
ictv_host_metadata = DataFrames.DataFrame(uCSV.read(ictv_metadata_file, header=1, delim='\t', typedetectrows=100))
ictv_host_metadata = ictv_host_metadata[.!isempty.(ictv_host_metadata[!, "taxid"]), :]
ictv_host_metadata[!, "taxid"] = parse.(Int, ictv_host_metadata[!, "taxid"])

viral_tax_ids = Set(Mycelia.list_subtaxa(10239))

Set{Int64} with 241049 elements:
  2956278
  656260
  1742422
  1427858
  2783550
  1599415
  300190
  2901373
  468417
  864692
  1149619
  2798297
  2946055
  2764961
  2827923
  1431632
  1733762
  1924399
  1613832
  2558856
  2822896
  1686811
  438298
  438823
  1347503
  ⋮ 

In [10]:
joint_top_hits = DataFrames.DataFrame()
ProgressMeter.@showprogress for sample_path in sample_paths[1:end]
    sample = basename(sample_path)
    blastn_directory = mkpath(joinpath(sample_path, "blastn"))
    assembled_fasta = joinpath(sample_path, "megahit", "final.contigs.fastg.gfa.fna")
    blast_file = joinpath(blastn_directory, basename(assembled_fasta) * ".blastn.$(db).$(blast_task).txt")
    this_blast_table = Mycelia.parse_blast_report(blast_file)
    if isempty(this_blast_table)
        continue
    else
        this_blast_table[!, "sample_id"] .= sample
        # bonferonni correction on raw tests
        this_blast_table[!, "evalue"] = this_blast_table[!, "evalue"] .* DataFrames.nrow(this_blast_table)
        
        # filter to top hits to avoid ballooning memory just to throw it away later
        this_top_hits = DataFrames.DataFrame()
        for gdf in DataFrames.groupby(this_blast_table, "query id")
            push!(this_top_hits, first(sort(gdf, "bit score", rev=true)))
        end
        append!(joint_top_hits, this_top_hits)
    end
end

Progress: 100%|█████████████████████████████████████████| Time: 0:11:52


In [11]:
taxids = unique(joint_top_hits[!, "subject tax id"])
taxid2name_map = Dict(row["taxid"] => row["tax_name"] for row in DataFrames.eachrow(Mycelia.taxids2lineage_name_and_rank(taxids)))
joint_top_hits[!, "subject tax name"] = map(taxid -> taxid2name_map[taxid], joint_top_hits[!, "subject tax id"])

# filter to good hits even after bonferroni correction
joint_top_hits = joint_top_hits[joint_top_hits[!, "evalue"] .<= 0.001, :]

# filter to viral only
viral_hits_df = joint_top_hits[map(x -> x in viral_tax_ids, joint_top_hits[!, "subject tax id"]), :]

viral_contigs_by_tool["blast"] = Dict{String, Set{String}}()
for gdf in DataFrames.groupby(viral_hits_df, "sample_id")
    sample_id = gdf[1, "sample_id"]
    viral_contigs_by_tool["blast"][sample_id] = Set()
    for row in DataFrames.eachrow(gdf)
        push!(viral_contigs_by_tool["blast"][sample_id], row["query id"])
    end
end
viral_contigs_by_tool["blast"]

00:48:33.737 [WARN] taxid 3086117 not found
00:48:33.737 [WARN] taxid 3066597 was deleted
00:48:33.737 [WARN] taxid 3076567 was deleted
00:48:33.737 [WARN] taxid 3076582 was deleted
00:48:33.738 [WARN] taxid 3076596 was deleted
00:48:33.737 [WARN] taxid 3066610 was deleted
00:48:33.738 [WARN] taxid 3068781 was deleted
00:48:33.738 [WARN] taxid 3066474 was deleted
00:48:33.738 [WARN] taxid 3063391 was deleted
00:48:33.739 [WARN] taxid 3068784 was deleted
00:48:33.739 [WARN] taxid 3076993 was deleted
00:48:33.739 [WARN] taxid 3084695 not found
00:48:33.739 [WARN] taxid 3092178 not found
00:48:33.739 [WARN] taxid 3083334 not found
00:48:33.739 [WARN] taxid 3093891 not found
00:48:33.739 [WARN] taxid 3082145 was deleted
00:48:33.739 [WARN] taxid 3101606 not found
00:48:33.739 [WARN] taxid 3067266 was deleted
00:48:33.739 [WARN] taxid 3086125 not found
00:48:33.739 [WARN] taxid 2996073 was deleted
00:48:33.740 [WARN] taxid 3096985 not found
00:48:33.740 [WARN] taxid 3076583 was deleted
00:4

Dict{String, Set{String}} with 213 entries:
  "SRR6399620" => Set(["67499", "6791", "10477", "11346", "18565", "2243", "556…
  "SRR6399522" => Set(["4304", "15513", "10467", "519", "19944", "30482", "1179…
  "SRR6399796" => Set(["12705", "12994", "15710", "1470", "12066", "3414", "113…
  "SRR6399686" => Set(["44880", "4209", "42469", "10249", "2590", "2579", "4369…
  "SRR6399461" => Set(["12486", "11346", "6086", "8913", "12388", "6831", "2052…
  "SRR6399715" => Set(["10467", "10684", "22328", "10825", "2504", "23232", "18…
  "SRR6399613" => Set(["9160", "12190", "8449", "4282", "1695", "7540", "12575"…
  "SRR6399716" => Set(["12486", "11346", "11803", "12754", "14918", "4497", "51…
  "SRR6399802" => Set(["2803", "30218", "3238", "34179", "30995", "519", "5422"…
  "SRR6399615" => Set(["2829", "1886", "3935", "4497", "5422", "10229", "5813",…
  "SRR6399464" => Set(["24624", "74129", "40188", "71250", "27096", "64525", "3…
  "SRR6400018" => Set(["24624", "33881", "5425", "1907", "26612",

In [12]:
db = "UniRef50"
uniref50_df = DataFrames.DataFrame()
ProgressMeter.@showprogress for sample_path in sample_paths
    sample_id = basename(sample_path)
    mmseqs_lca_file = joinpath(sample_path, "mmseqs_easy_taxonomy", "final.contigs.fastg.gfa.fna.mmseqs_easy_taxonomy.$(db)_lca.tsv")
    mmseqs_lca_table = Mycelia.parse_mmseqs_easy_taxonomy_lca_tsv(mmseqs_lca_file)
    mmseqs_lca_table[!, "sample_id"] .= sample_id
    append!(uniref50_df, mmseqs_lca_table)
end
uniref50_df
uniref50_viral_df = uniref50_df[map(x -> x in viral_tax_ids, uniref50_df[!, "taxon_id"]), :]

viral_contigs_by_tool[db] = Dict{String, Set{String}}()
for gdf in DataFrames.groupby(uniref50_viral_df, "sample_id")
    sample_id = gdf[1, "sample_id"]
    viral_contigs_by_tool[db][sample_id] = Set()
    for row in DataFrames.eachrow(gdf)
        push!(viral_contigs_by_tool[db][sample_id], string(row["contig_id"]))
    end
end
viral_contigs_by_tool[db]

Progress: 100%|█████████████████████████████████████████| Time: 0:01:00


Dict{String, Set{String}} with 196 entries:
  "SRR6399620" => Set(["21030", "58575", "40326", "54792", "71900", "23976", "2…
  "SRR6399522" => Set(["4503", "32284", "7890", "38656", "34901", "31490", "344…
  "SRR6399796" => Set(["15763", "10398"])
  "SRR6399686" => Set(["28296", "14741", "56873", "28329", "55585", "10656", "7…
  "SRR6399461" => Set(["11505"])
  "SRR6399715" => Set(["14228", "24056", "8240", "34678", "34655", "27242", "28…
  "SRR6399613" => Set(["15644", "14054"])
  "SRR6399716" => Set(["13272"])
  "SRR6399802" => Set(["28406", "18527", "28037", "18855", "12297", "4537"])
  "SRR6399615" => Set(["5422", "10127"])
  "SRR6399464" => Set(["29457", "32202", "53922", "73334", "22334", "49336", "3…
  "SRR6400018" => Set(["8105", "6160", "31294", "20562", "33407", "14446", "591…
  "SRR6399728" => Set(["11792", "9785", "19471"])
  "SRR6399569" => Set(["3540", "36279", "10280", "4953", "18712", "14359", "261…
  "SRR6399871" => Set(["7922"])
  "SRR6399460" => Set(["5972", "8067", 

In [13]:
db = "UniRef90"
uniref90_df = DataFrames.DataFrame()

ProgressMeter.@showprogress for sample_path in sample_paths
    sample_id = basename(sample_path)
    mmseqs_lca_file = joinpath(sample_path, "mmseqs_easy_taxonomy", "final.contigs.fastg.gfa.fna.mmseqs_easy_taxonomy.$(db)_lca.tsv")
    mmseqs_lca_table = Mycelia.parse_mmseqs_easy_taxonomy_lca_tsv(mmseqs_lca_file)
    mmseqs_lca_table[!, "sample_id"] .= sample_id
    append!(uniref90_df, mmseqs_lca_table)
end
uniref90_df
uniref90_viral_df = uniref90_df[map(x -> x in viral_tax_ids, uniref90_df[!, "taxon_id"]), :]

viral_contigs_by_tool[db] = Dict{String, Set{String}}()
for gdf in DataFrames.groupby(uniref90_viral_df, "sample_id")
    sample_id = gdf[1, "sample_id"]
    viral_contigs_by_tool[db][sample_id] = Set()
    for row in DataFrames.eachrow(gdf)
        push!(viral_contigs_by_tool[db][sample_id], string(row["contig_id"]))
    end
end
viral_contigs_by_tool[db]

Progress: 100%|█████████████████████████████████████████| Time: 0:01:10


Dict{String, Set{String}} with 204 entries:
  "SRR6399620" => Set(["21030", "58575", "40326", "54792", "21839", "20456", "5…
  "SRR6399522" => Set(["32284", "26514", "8409", "19331", "35816", "5227", "175…
  "SRR6399796" => Set(["15763", "10398"])
  "SRR6399686" => Set(["35488", "26650", "56873", "28329", "28296", "54674", "1…
  "SRR6399461" => Set(["11505", "6146"])
  "SRR6399715" => Set(["33095", "24056", "14228", "8240", "34678", "34655", "28…
  "SRR6399613" => Set(["15644", "14054", "9956"])
  "SRR6399716" => Set(["15781", "7990", "13272"])
  "SRR6399802" => Set(["28406", "18527", "12804", "15545", "20767", "18855", "2…
  "SRR6399615" => Set(["5422", "10127"])
  "SRR6399464" => Set(["73334", "68375", "40745", "31585", "10089", "33938", "2…
  "SRR6400018" => Set(["8105", "21303", "31294", "6160", "20562", "30724", "865…
  "SRR6399728" => Set(["9785", "19471"])
  "SRR6399569" => Set(["15296", "3540", "12041", "36279", "12175", "18712", "49…
  "SRR6399871" => Set(["7922", "7209", "137

In [14]:
db = "UniRef100"
uniref100_df = DataFrames.DataFrame()

ProgressMeter.@showprogress for sample_path in sample_paths
    sample_id = basename(sample_path)
    mmseqs_lca_file = joinpath(sample_path, "mmseqs_easy_taxonomy", "final.contigs.fastg.gfa.fna.mmseqs_easy_taxonomy.$(db)_lca.tsv")
    mmseqs_lca_table = Mycelia.parse_mmseqs_easy_taxonomy_lca_tsv(mmseqs_lca_file)
    mmseqs_lca_table[!, "sample_id"] .= sample_id
    append!(uniref100_df, mmseqs_lca_table)
end
uniref100_df
uniref100_viral_df = uniref100_df[map(x -> x in viral_tax_ids, uniref100_df[!, "taxon_id"]), :]

viral_contigs_by_tool[db] = Dict{String, Set{String}}()
for gdf in DataFrames.groupby(uniref100_viral_df, "sample_id")
    sample_id = gdf[1, "sample_id"]
    viral_contigs_by_tool[db][sample_id] = Set()
    for row in DataFrames.eachrow(gdf)
        push!(viral_contigs_by_tool[db][sample_id], string(row["contig_id"]))
    end
end
viral_contigs_by_tool[db]

Progress: 100%|█████████████████████████████████████████| Time: 0:01:09


Dict{String, Set{String}} with 204 entries:
  "SRR6399620" => Set(["21030", "25060", "58575", "40326", "54792", "21839", "2…
  "SRR6399522" => Set(["32284", "26514", "8409", "19331", "35816", "5227", "175…
  "SRR6399796" => Set(["15763", "10398"])
  "SRR6399686" => Set(["26650", "35488", "56873", "28329", "28296", "10624", "5…
  "SRR6399461" => Set(["11505", "6146"])
  "SRR6399715" => Set(["33095", "14228", "8240", "24056", "34678", "34655", "28…
  "SRR6399613" => Set(["15644", "11681", "14054", "9956"])
  "SRR6399716" => Set(["15781", "7990", "14997", "1750", "13272", "4734"])
  "SRR6399802" => Set(["28406", "18527", "28037", "15545", "18855", "12297", "2…
  "SRR6399615" => Set(["5422", "10127"])
  "SRR6399464" => Set(["73334", "68375", "40745", "74632", "31585", "10089", "3…
  "SRR6400018" => Set(["8105", "21303", "6160", "31294", "30724", "20562", "848…
  "SRR6399728" => Set(["9785", "19471"])
  "SRR6399569" => Set(["12041", "3540", "12175", "36279", "15401", "18712", "49…
  "SRR639

In [15]:
viral_contigs_by_tool

Dict{String, Dict{String, Set{String}}} with 5 entries:
  "UniRef50"  => Dict("SRR6399620"=>Set(["21030", "58575", "40326", "54792", "7…
  "blast"     => Dict("SRR6399620"=>Set(["67499", "6791", "10477", "11346", "18…
  "UniRef100" => Dict("SRR6399620"=>Set(["21030", "25060", "58575", "40326", "5…
  "genomad"   => Dict("SRR6399620"=>Set(["25060", "58575", "21490", "57482", "3…
  "UniRef90"  => Dict("SRR6399620"=>Set(["21030", "58575", "40326", "54792", "2…

In [16]:
ordered_tools = 
["genomad",
"blast",
"UniRef50",
"UniRef90",
"UniRef100"]

ordered_samples = basename.(sample_paths)

213-element Vector{String}:
 "SRR6399459"
 "SRR6399460"
 "SRR6399461"
 "SRR6399462"
 "SRR6399464"
 "SRR6399485"
 "SRR6399486"
 "SRR6399487"
 "SRR6399488"
 "SRR6399490"
 "SRR6399491"
 "SRR6399492"
 "SRR6399494"
 ⋮
 "SRR6399977"
 "SRR6399986"
 "SRR6399988"
 "SRR6399991"
 "SRR6400014"
 "SRR6400015"
 "SRR6400017"
 "SRR6400018"
 "SRR6400019"
 "SRR6400022"
 "SRR6400023"
 "SRR6400024"

In [17]:
unanimous_hits = Dict{String, Set{String}}()
for sample in ordered_samples
    unanimous_hits[sample] = Set(viral_contigs_by_tool["genomad"][sample])
    for other_tool in setdiff(ordered_tools, "genomad")
        # @show other_tool
        unanimous_hits[sample] = intersect(unanimous_hits[sample], viral_contigs_by_tool["genomad"][sample])
    end
end
unanimous_hits

Dict{String, Set{String}} with 213 entries:
  "SRR6399620" => Set(["25060", "58575", "21490", "57482", "33110", "43681", "1…
  "SRR6399522" => Set(["35351", "10657", "39489", "35171", "30061", "39970", "2…
  "SRR6399796" => Set(["4659", "15743", "15859", "2360", "15763", "12792", "112…
  "SRR6399686" => Set(["28931", "20278", "38091", "17527", "41020", "45034", "2…
  "SRR6399461" => Set(["6086", "9587", "4230", "20677", "1651", "14090", "3973"…
  "SRR6399715" => Set(["34950", "32420", "26563", "18322", "25183", "33125", "2…
  "SRR6399613" => Set(["12960", "9620", "3747", "1848", "8951", "9945", "10214"…
  "SRR6399716" => Set(["9687", "8891", "540", "6327", "15778", "1750", "263", "…
  "SRR6399802" => Set(["34461", "5822", "23752", "28306", "7925", "22826", "302…
  "SRR6399615" => Set(["3478", "908", "3343", "9449", "6666", "8771", "4800", "…
  "SRR6399464" => Set(["36948", "5790", "34631", "52460", "53243", "22797", "74…
  "SRR6400018" => Set(["4304", "35674", "36966", "17750", "30029"

In [105]:
# "Caudoviricetes sp." => 390

In [106]:
# sort(collect(StatsBase.countmap(unanimous_blast_hits[!, "subject tax name"])), by=x->x[2], rev=true)

In [107]:
# blast
# unanimous_blast_hits = viral_hits_df[[row["query id"] in unanimous_hits[row["sample_id"]] for row in DataFrames.eachrow(viral_hits_df)], :]
# unanimous_blast_host_hits = unanimous_blast_host_hits[map(x -> occursin("papillomavirus", x), unanimous_blast_host_hits[!, "subject tax name"]), :]

In [119]:
blast_taxon_counts = sort(collect(StatsBase.countmap(unanimous_blast_hits[!, "subject tax name"])), by=x->x[2], rev=true)
blast_taxon_relative_proportions = [x => y/sum(last.(blast_taxon_counts)) for (x, y) in blast_taxon_counts]
round(sum(last.(filter(x -> occursin(r"caudovir"i, x[1]), blast_taxon_relative_proportions))) * 100, digits=3)

24.296

In [120]:
# uniref100_viral_df
unanimous_uniref100_hits = uniref100_viral_df[[string(row["contig_id"]) in unanimous_hits[row["sample_id"]] for row in DataFrames.eachrow(uniref100_viral_df)], :]
uniref100_taxon_counts = sort(collect(StatsBase.countmap(unanimous_uniref100_hits[!, "taxon_name"])), by=x->x[2], rev=true)
uniref100_taxon_relative_proportions = [x => y/sum(last.(uniref100_taxon_counts)) for (x, y) in uniref100_taxon_counts]
round(sum(last.(filter(x -> occursin(r"caudovir"i, x[1]), uniref100_taxon_relative_proportions))) * 100, digits=3)

25.998

In [122]:
# uniref90_viral_df
unanimous_uniref90_hits = uniref90_viral_df[[string(row["contig_id"]) in unanimous_hits[row["sample_id"]] for row in DataFrames.eachrow(uniref90_viral_df)], :]
uniref90_taxon_counts = sort(collect(StatsBase.countmap(unanimous_uniref90_hits[!, "taxon_name"])), by=x->x[2], rev=true)
uniref90_taxon_relative_proportions = [x => y/sum(last.(uniref90_taxon_counts)) for (x, y) in uniref90_taxon_counts]
round(sum(last.(filter(x -> occursin(r"caudovir"i, x[1]), uniref90_taxon_relative_proportions))) * 100, digits=3)

30.662

In [123]:
# uniref50_viral_df
unanimous_uniref50_hits = uniref50_viral_df[[string(row["contig_id"]) in unanimous_hits[row["sample_id"]] for row in DataFrames.eachrow(uniref50_viral_df)], :]
uniref50_taxon_counts = sort(collect(StatsBase.countmap(unanimous_uniref50_hits[!, "taxon_name"])), by=x->x[2], rev=true)
uniref50_taxon_relative_proportions = [x => y/sum(last.(uniref50_taxon_counts)) for (x, y) in uniref50_taxon_counts]
round(sum(last.(filter(x -> occursin(r"caudovir"i, x[1]), uniref50_taxon_relative_proportions))) * 100, digits=3)

28.718

In [131]:
contigs_of_interest = unique(vcat(
    DataFrames.rename(unanimous_blast_hits[map(x -> occursin(r"caudovir"i, x), unanimous_blast_hits[!, "subject tax name"]), ["sample_id", "query id"]], "query id" => "contig_id"),
    unanimous_uniref100_hits[map(x -> occursin(r"caudovir"i, x), unanimous_uniref100_hits[!, "taxon_name"]), ["sample_id", "contig_id"]],
    unanimous_uniref90_hits[map(x -> occursin(r"caudovir"i, x), unanimous_uniref90_hits[!, "taxon_name"]), ["sample_id", "contig_id"]],
    unanimous_uniref50_hits[map(x -> occursin(r"caudovir"i, x), unanimous_uniref50_hits[!, "taxon_name"]), ["sample_id", "contig_id"]]
))

Row,sample_id,contig_id
,String,Any
1,SRR6399461,19066
2,SRR6399461,2188
3,SRR6399461,4230
4,SRR6399464,44462
5,SRR6399464,48520
6,SRR6399464,69238
7,SRR6399485,14937
8,SRR6399486,10662
9,SRR6399486,1480


In [176]:
# extract contigs into a single fasta file
fasta_records = []
ProgressMeter.@showprogress for gdf in DataFrames.groupby(contigs_of_interest, "sample_id")
    contig_ids = Set(gdf[!, "contig_id"])
    sample_id = gdf[1, "sample_id"]
    sample_path = joinpath(data_dir, "SRA", sample_id)
    fasta_path = joinpath(sample_path, "megahit", "final.contigs.fastg.gfa.fna")
    open(fasta_path) do io
        fastx_io = FASTX.FASTA.Reader(io)
        for record in fastx_io
            if FASTX.identifier(record) in contig_ids
                push!(fasta_records, FASTX.FASTA.Record(sample_id * "__" * FASTX.identifier(record), FASTX.sequence(record)))
            end
        end
        close(fastx_io)
    end
end
fasta_records

Progress: 100%|█████████████████████████████████████████| Time: 0:01:54


509-element Vector{Any}:
 FASTX.FASTA.Record:
  description: "SRR6399461__19066"
     sequence: "ATAGCATTATTTGTTATAATAGGGGGGATATTATTATTA…"
 FASTX.FASTA.Record:
  description: "SRR6399461__2188"
     sequence: "CAAAATGAAGAAAGAATATTATTTGCTGATTTGTCCATA…"
 FASTX.FASTA.Record:
  description: "SRR6399461__4230"
     sequence: "GAATATGATTTCAAAAATAGCTTCCTTGAAAGGAGGGGC…"
 FASTX.FASTA.Record:
  description: "SRR6399464__44462"
     sequence: "TATTGCACCAAGTAATGCATCGTATAAAGCTGTAAGTTT…"
 FASTX.FASTA.Record:
  description: "SRR6399464__48520"
     sequence: "AACTAAGTATAAAGAATCATTTAAAAGTGCAAATCCTCG…"
 FASTX.FASTA.Record:
  description: "SRR6399464__69238"
     sequence: "GTTAAGATTTATGGTAAACAATTCACTGCTAAAGATATT…"
 FASTX.FASTA.Record:
  description: "SRR6399485__14937"
     sequence: "GGTAGGTGTTTCCGCTGATGCTGTAGGTGGCTGTGCCGT…"
 FASTX.FASTA.Record:
  description: "SRR6399486__10662"
     sequence: "AACGTAATTTTATTAAATTAGTACAAGTAGACAAAAAAG…"
 FASTX.FASTA.Record:
  description: "SRR6399486__1480"
     seque

In [177]:
fasta_records = sort(fasta_records, by=x->length(FASTX.sequence(x)), rev=true)

509-element Vector{Any}:
 FASTX.FASTA.Record:
  description: "SRR6399523__6674"
     sequence: "TAAAATGCTTCATTTGTAATGAAAAATTAAATCATAATC…"
 FASTX.FASTA.Record:
  description: "SRR6399523__924"
     sequence: "CACATTTACATAAGGGGTAAGAATCATGAAACATATATT…"
 FASTX.FASTA.Record:
  description: "SRR6399728__3111"
     sequence: "TATAGTTACATTTCCTGTGGAATTTGCATTTGCTTTAAA…"
 FASTX.FASTA.Record:
  description: "SRR6399588__6685"
     sequence: "TTGGTCAAGGAATTGCAAATGGTTTAGGTGGAAATGCTA…"
 FASTX.FASTA.Record:
  description: "SRR6399955__93301"
     sequence: "CTGACGCATAGCCATGTGGTGTCGGTGCGGCTCATGAGA…"
 FASTX.FASTA.Record:
  description: "SRR6399871__3286"
     sequence: "GCAACGTTTTGCATCGCCTGATACTTCTTCACCGACGAT…"
 FASTX.FASTA.Record:
  description: "SRR6399718__21250"
     sequence: "CTTGAAACCCCCCTATTTGCCAATAAAAATGTTGACCCA…"
 FASTX.FASTA.Record:
  description: "SRR6399773__39766"
     sequence: "ACGAGATAATAAAAAGGGTATTCCATTAAGAATATTTTT…"
 FASTX.FASTA.Record:
  description: "SRR6399516__19941"
     sequence

In [180]:
p = StatsPlots.histogram(
    length.(FASTX.sequence.(fasta_records)),
    bins=100,
    label=missing,
    xlabel = "contig length",
    ylabel = "# of contigs",
    title = "high confidence caudovirales contigs"
)
StatsPlots.vline!(p, [2000], label = "2Kb")
StatsPlots.savefig(p, joinpath(results_dir, "high-confidence-caudovirales-contigs.fna.2k-filtered.png"))

"/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/results/high-confidence-caudovirales-contigs.fna.2k-filtered.png"

In [ ]:
filtered_fasta_records = filter(x -> length(FASTX.sequence(x)) >= 2000, fasta_records)

caudovirales_fasta = joinpath(results_dir, "high-confidence-caudovirales-contigs.fna")
open(caudovirales_fasta, "w") do io
    fastx_io = FASTX.FASTA.Writer(io)
    for record in filtered_fasta_records
        write(fastx_io, record)
    end
    close(fastx_io)
end

bgzipped_caudovirales_fasta = caudovirales_fasta * ".gz"
if isfile(bgzipped_caudovirales_fasta)
    rm(bgzipped_caudovirales_fasta)
end
run(`conda run --live-stream -n samtools bgzip $(caudovirales_fasta)`)
run(`conda run --live-stream -n samtools samtools faidx $(bgzipped_caudovirales_fasta)`)
# function fasta_to_fasta_alignment(ref, query)
#     run(pipeline(`conda run --live-stream -n minimap2 minimap2 -ax asm5 $(ref) $(query)`, ref * ".$(basename(query)).sam"))
# end
# # ref = joinpath(data_dir, "reference-genomes/GCF_001274345.1_ViralProj293723_genomic.fna")
# # fasta_to_fasta_alignment(ref, bgzipped_hpv_fasta)

# # not sensitive to segment length or 
pggb_outdir = joinpath(results_dir, "pggb_caudovirales")
# haplotypes = 2
haplotypes = Int(ceil(sqrt(length(filtered_fasta_records))))
 #  
# run(`conda run --live-stream -n pggb pggb -i $(bgzipped_caudovirales_fasta) -o $(pggb_outdir) -t 4 -n $(haplotypes) -p 70 -s 100 -l 300`)
run(`conda run --live-stream -n pggb pggb -i $(bgzipped_caudovirales_fasta) -o $(pggb_outdir) -t 4 -n $(haplotypes)`)
# # -t 4 

In [189]:
filter(x -> occursin(r"\.fasta$", x), readdir(joinpath(data_dir, "exposome_data", "joint-reads"), join=true))

24-element Vector{String}:
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/exposome_data/joint-reads/m64283e_231026_230505.bcAd1001T--bcAd1001T.bam.fasta"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/exposome_data/joint-reads/m64283e_231026_230505.bcAd1002T--bcAd1002T.bam.fasta"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/exposome_data/joint-reads/m64283e_231026_230505.bcAd1003T--bcAd1003T.bam.fasta"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/exposome_data/joint-reads/m64283e_231026_230505.bcAd1004T--bcAd1004T.bam.fasta"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/exposome_data/joint-reads/m64283e_231026_230505.bcAd1006T--bcAd1006T.bam.fasta"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/exposome_data/joint-reads/m64283e_23102

In [191]:
tmp_fasta = tempname() * ".fna"
long_read_fastas = filter(x -> occursin(r"\.fasta$", x), readdir(joinpath(data_dir, "exposome_data", "joint-reads"), join=true))
Mycelia.merge_fasta_files(long_read_fastas, tmp_fasta)

"/local/scratch/cjprybol/slrmtmp.42296384/jl_9vQATZ.fna"

In [193]:
# run(`minimap2 -a ref.fa $(tmp_fasta) > alignment.sam`)
query = tmp_fasta
outsam = ref * ".$(basename(query)).sam"
run(pipeline(`conda run --live-stream -n minimap2 minimap2 -a $(ref) $(query)`, outsam))

"/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/reference-genomes/GCF_001274345.1_ViralProj293723_genomic.fna.jl_9vQATZ.fna.sam"

In [194]:
filtered_sam = replace(outsam, ".sam" => ".filtered.sam")

"/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/reference-genomes/GCF_001274345.1_ViralProj293723_genomic.fna.jl_9vQATZ.fna.filtered.sam"

In [195]:
run(pipeline(`conda run --live-stream -n samtools samtools view -F 4 $(outsam)`, filtered_sam))

Process(`conda run --live-stream -n samtools samtools view -F 4 /oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/reference-genomes/GCF_001274345.1_ViralProj293723_genomic.fna.jl_9vQATZ.fna.sam`, ProcessExited(0))

In [72]:
# current_host = "host_is_human"
# host_viral_tax_ids = Set(ncbi_host_metadata[ncbi_host_metadata[!, current_host] .== true, "taxid"])

# # jumping straight into HPV, since that's the only hit shared across DNA and Protein
# # blast
# unanimous_blast_hits = viral_hits_df[[row["query id"] in unanimous_hits[row["sample_id"]] for row in DataFrames.eachrow(viral_hits_df)], :]
# unanimous_blast_host_hits = unanimous_blast_hits[map(x -> x in host_viral_tax_ids, unanimous_blast_hits[!, "subject tax id"]), :]
# unanimous_blast_host_hits = unanimous_blast_host_hits[map(x -> occursin("papillomavirus", x), unanimous_blast_host_hits[!, "subject tax name"]), :]

# # uniref100_viral_df
# unanimous_uniref100_hits = uniref100_viral_df[[string(row["contig_id"]) in unanimous_hits[row["sample_id"]] for row in DataFrames.eachrow(uniref100_viral_df)], :]
# unanimous_uniref100_host_hits = unanimous_uniref100_hits[map(x -> x in host_viral_tax_ids, unanimous_uniref100_hits[!, "taxon_id"]), :]
# unanimous_uniref100_host_hits = unanimous_uniref100_hits[map(x -> x in host_viral_tax_ids, unanimous_uniref100_hits[!, "taxon_name"]), :]

# # uniref90_viral_df
# unanimous_uniref90_hits = uniref90_viral_df[[string(row["contig_id"]) in unanimous_hits[row["sample_id"]] for row in DataFrames.eachrow(uniref90_viral_df)], :]
# unanimous_uniref90_host_hits = unanimous_uniref90_hits[map(x -> x in host_viral_tax_ids, unanimous_uniref90_hits[!, "taxon_id"]), :]
# unanimous_uniref90_host_hits = unanimous_uniref90_host_hits[map(x -> x in host_viral_tax_ids, unanimous_uniref90_host_hits[!, "taxon_name"]), :]

# # uniref50_viral_df
# unanimous_uniref50_hits = uniref50_viral_df[[string(row["contig_id"]) in unanimous_hits[row["sample_id"]] for row in DataFrames.eachrow(uniref50_viral_df)], :]
# unanimous_uniref50_host_hits = unanimous_uniref50_hits[map(x -> x in host_viral_tax_ids, unanimous_uniref50_hits[!, "taxon_id"]), :]
# unanimous_uniref50_host_hits = unanimous_uniref50_host_hits[map(x -> x in host_viral_tax_ids, unanimous_uniref50_host_hits[!, "taxon_name"]), :]

# contigs_of_interest = unique(vcat(
#     DataFrames.rename(unanimous_blast_host_hits[!, ["sample_id", "query id"]], "query id" => "contig_id"),
#     unanimous_uniref100_host_hits[!, ["sample_id", "contig_id"]],
#     unanimous_uniref90_host_hits[!, ["sample_id", "contig_id"]],
#     unanimous_uniref50_host_hits[!, ["sample_id", "contig_id"]],
# ))

# # extract contigs into a single fasta file
# fasta_records = []
# for gdf in DataFrames.groupby(contigs_of_interest, "sample_id")
#     contig_ids = Set(gdf[!, "contig_id"])
#     sample_id = gdf[1, "sample_id"]
#     sample_path = joinpath(data_dir, "SRA", sample_id)
#     fasta_path = joinpath(sample_path, "megahit", "final.contigs.fastg.gfa.fna")
#     open(fasta_path) do io
#         fastx_io = FASTX.FASTA.Reader(io)
#         for record in fastx_io
#             if FASTX.identifier(record) in contig_ids
#                 push!(fasta_records, record)
#             end
#         end
#         close(fastx_io)
#     end
# end
# fasta_records

# hpv_fasta = joinpath(results_dir, "high-confidence-hpv-contigs.fna")
# open(hpv_fasta, "w") do io
#     fastx_io = FASTX.FASTA.Writer(io)
#     for record in fasta_records
#         write(fastx_io, record)
#     end
#     close(fastx_io)
# end

# bgzipped_hpv_fasta = hpv_fasta * ".gz"
# if isfile(bgzipped_hpv_fasta)
#     rm(bgzipped_hpv_fasta)
# end
# run(`conda run --live-stream -n samtools bgzip $(hpv_fasta)`)
# run(`conda run --live-stream -n samtools samtools faidx $(bgzipped_hpv_fasta)`)
# function fasta_to_fasta_alignment(ref, query)
#     run(pipeline(`conda run --live-stream -n minimap2 minimap2 -ax asm5 $(ref) $(query)`, ref * ".$(basename(query)).sam"))
# end

# ref = joinpath(data_dir, "reference-genomes/GCF_001274345.1_ViralProj293723_genomic.fna")
# fasta_to_fasta_alignment(ref, bgzipped_hpv_fasta)

# # not sensitive to segment length or 
# pggb_outdir = joinpath(results_dir, "pggb_hpv")
# haplotypes = 2
# run(`conda run --live-stream -n pggb pggb -i $(bgzipped_hpv_fasta) -o $(pggb_outdir) -n $(haplotypes) --map-pct-id 70 --segment-length 100`)
# # -t 4 

In [ ]:
#make and visualize pangenome
# conda create --channel conda-forge --channel bioconda --channel defaults --strict-channel-priority --name pggb pggb





In [ ]:
#map long reads to them